In [1]:
import networkx as nx 
import numpy as np
import matplotlib.pyplot as plt

from regain.datasets.multi_class import make_multiclass_dataset

# Data generation

In [2]:
res, binaries = make_multiclass_dataset(n_samples=100, n_dim_obs=20, 
                                                             n_classes=5,_type='erdos-renyi', n_edges=1)

In [3]:
def multi_class_kernel(n_classes=5, beta=1):
    kernel = np.ones((n_classes, n_classes))*beta
    np.fill_diagonal(kernel, 1)
    return kernel

In [4]:
kernel = multi_class_kernel(beta=10)

# Fitting

In [5]:
from regain.covariance.kernel_time_graphical_lasso_ import KernelTimeGraphicalLasso
from regain.generalized_temporal_linear_model.poisson import TemporalPoissonModel
from regain.generalized_temporal_linear_model.ising import TemporalIsingModel
from regain.utils import structure_error

In [7]:
tglk = KernelTimeGraphicalLasso(alpha=8, kernel=kernel, psi='l1')
tigmk = TemporalIsingModel(alpha=8, kernel=kernel, psi='l1')
tpgmk = TemporalPoissonModel(alpha=8, kernel=kernel, psi='l1')

from sklearn.model_selection import StratifiedShuffleSplit
import warnings 

warnings.simplefilter('ignore')
sizes = [5, 10, 50, 100]
res_g = {s:[] for s in sizes}
res_i = {s:[] for s in sizes}
res_p = {s:[] for s in sizes}
for s in sizes:
    sss = StratifiedShuffleSplit(n_splits=3, train_size=s)
    for train, _ in sss.split(res['gaussian']['X'],res['gaussian']['y']):
        tglk.fit(res['gaussian']['X'][train, :],res['gaussian']['y'][train])
        res_g[s].append(tglk.precision_)